# Clusterização de clientes de instituição financeira
Neste projeto, eu construirei um modelo de machine learning para agrupar clientes de uma instituição financeira baseado nos seus perfis de gastos, de forma que eles possam ter mais controle e gestão sobre suas finanças através de um novo produto que está sendo desenvolvido. Esta é uma tarefa de aprendizado não supervisionado, em que temos um conjunto de dados sem rótulos.

Utilizarei o seguinte pipeline de solução, baseado no framework CRISP-DM:
1. Definir o problema de negócio.
2. Coletar os dados e obter uma visão geral deles.
3. Explorar os dados (EDA) e realizar feature engineering.
4. Limpeza e pré-processamento dos dados.
5. Agrupar clientes em clusters utilizando algoritmos de machine learning.
6. Interpretar os resultados, analisando os grupos criados e obtendo personas. Então, fornecer insights de negócio para auxiliar a tomada de decisão pela empresa.

Cada passo será abordado em detalhes abaixo, explicando o que será feito e o porquê de cada decisão tomada.

# 1. Problema de negócio
Uma empresa do setor financeiro deseja agrupar clientes baseado nos seus perfis de gastos, de forma que eles possam ter mais controle e gestão sobre suas finanças através de um produto que está sendo desenvolvido.

Agrupar os clientes de acordo com o perfil de gastos pode ser extremamente benéfico para uma empresa que desenvolve um produto de gerenciamento de gastos. Entre os motivos para isso, estão:

1. Personalização de Serviços: Agrupar clientes com base em seus perfis de gastos permite que a empresa personalize seus serviços e recomendações. Clientes com comportamentos de gastos semelhantes podem receber orientações específicas e ofertas personalizadas que atendam às suas necessidades individuais.

2. Segmentação de Mercado: A segmentação de clientes com base em seus padrões de gastos ajuda a empresa a identificar grupos de clientes com características e preferências semelhantes. Isso pode levar a estratégias de marketing mais eficazes, direcionadas a públicos-alvo específicos, aumentando as chances de conversão.

3. Melhoria na Experiência do Cliente: Ao entender melhor como diferentes grupos de clientes gastam seu dinheiro, a empresa pode ajustar a interface e as funcionalidades do produto para atender às necessidades de cada grupo. Isso melhora a experiência do cliente e a satisfação geral.

4. Aprimoramento do Desenvolvimento de Produtos: Ao analisar os dados dos diferentes grupos de clientes, a empresa pode obter insights valiosos para o desenvolvimento de novos recursos ou produtos. Isso ajuda a garantir que o produto evolua de acordo com as necessidades reais dos clientes.

### Importando as bibliotecas necessárias

In [138]:
# Manipulação e visualização de dados.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 2. Entendimento geral dos dados
- Os dados são fictícios e contêm informações de pessoas, contas e transações. 
- A base de transações possui registros detalhados de transações bancárias e de cartão de crédito.
- As bases se relacionam por meio de identificadores. 

In [139]:
accounts = pd.read_csv('data/accounts.csv')
person = pd.read_csv('data/person.csv')
transactions = pd.read_csv('data/transactions.csv')

Antes de tudo, vamos dar uma olhada nos 3 datasets, de forma que possamos uni-los através da função "merge" do Pandas.

In [140]:
accounts.head()

,id,type,subtype,balance,currency_code,name,person_id,account_id,date
0,1968bd13-dd25-4f88-bccc-5cff92d6811b,BANK,SAVINGS_ACCOUNT,3826.72,BRL,Caixinha,644f5d18-5bfa-45f9-8aa7-401c05b3b9f2,1968bd13-dd25-4f88-bccc-5cff92d6811b,2023-09-14 14:03:16
1,978f92cf-8d5b-437a-a046-cdaa9105624c,CREDIT,CREDIT_CARD,6311.67,BRL,Cartão de Crédito,644f5d18-5bfa-45f9-8aa7-401c05b3b9f2,978f92cf-8d5b-437a-a046-cdaa9105624c,2023-09-15 15:50:21
2,dcd96f70-8b16-42b0-a944-afec5d8305ce,CREDIT,CREDIT_CARD,7482.25,BRL,Cartão de Crédito,644f5d18-5bfa-45f9-8aa7-401c05b3b9f2,dcd96f70-8b16-42b0-a944-afec5d8305ce,2023-09-09 03:38:49
3,6c2a44fe-2ba7-4626-aaff-f244a0647eae,BANK,SAVINGS_ACCOUNT,5740.29,BRL,Caixinha,644f5d18-5bfa-45f9-8aa7-401c05b3b9f2,6c2a44fe-2ba7-4626-aaff-f244a0647eae,2023-09-12 14:21:21
4,abb359a2-3802-48e7-976b-8671e1de0884,BANK,CHECKING_ACCOUNT,5136.49,BRL,Conta Corrente,644f5d18-5bfa-45f9-8aa7-401c05b3b9f2,abb359a2-3802-48e7-976b-8671e1de0884,2023-09-12 04:10:04


In [141]:
person.head()

,id,name,suitability
0,644f5d18-5bfa-45f9-8aa7-401c05b3b9f2,Ana Julia Santos,conservador
1,01018c18-ecb5-4cb6-b16f-bece97935974,Emilly Moura,moderado-agressivo
2,65d3336e-dafd-4663-8a53-97792c847074,Luiz Miguel Caldeira,conservador
3,6a33c027-13f2-4fbf-8bbb-74e6438d11c8,Sarah Souza,moderado-conservador
4,55252085-c5d6-4b7d-b221-fd3ad76ba8e5,Samuel Oliveira,moderado-agressivo


In [142]:
transactions.head()

,id,currency_code,amount,date,category,account_id
0,b7ebd81b-98f7-47ab-91f3-52bc80be57a6,BRL,-498.71,2023-07-20 03:57:57,Educação,1968bd13-dd25-4f88-bccc-5cff92d6811b
1,b8d83679-02ab-4241-b15d-11e9c0033864,BRL,-265.00,2023-07-23 05:17:06,Outras saídas,1968bd13-dd25-4f88-bccc-5cff92d6811b
2,99cab263-da5c-4706-9f2c-92cb779acaf7,BRL,-18.95,2023-06-26 05:54:25,Transporte,1968bd13-dd25-4f88-bccc-5cff92d6811b
3,5a78f1f0-f541-440f-9d3d-c8c21373ffc8,BRL,-262.70,2023-06-21 00:34:08,Outras saídas,1968bd13-dd25-4f88-bccc-5cff92d6811b
4,fc4e9552-bb41-4757-8c16-ff2c584ee01b,BRL,-179.44,2023-08-05 01:09:42,"Imposto, juros e multa",1968bd13-dd25-4f88-bccc-5cff92d6811b


Olhando as dimensõs dos três datasets.

In [143]:
accounts.shape, person.shape, transactions.shape

((7569, 9), (2531, 3), (400110, 6))

Considerando que a coluna id em person corresponde ao person_id em accounts, vou realizar a união desses dois. Então, uma vez que account_id é a chave comum entre accounts e transactions, unirei novamente, obtendo o dataset final.

In [144]:
merged_df = pd.merge(accounts, person, left_on='person_id', right_on='id', how='inner')
df.head()

,account_type,account_subtype,account_balance,account_name,account_date,person_name,person_suitability,transaction_amount,transaction_date,transaction_category
0,BANK,SAVINGS_ACCOUNT,3826.72,Caixinha,2023-09-14 14:03:16,Ana Julia Santos,conservador,-498.71,2023-07-20 03:57:57,Educação
1,BANK,SAVINGS_ACCOUNT,3826.72,Caixinha,2023-09-14 14:03:16,Ana Julia Santos,conservador,-265.00,2023-07-23 05:17:06,Outras saídas
2,BANK,SAVINGS_ACCOUNT,3826.72,Caixinha,2023-09-14 14:03:16,Ana Julia Santos,conservador,-18.95,2023-06-26 05:54:25,Transporte
3,BANK,SAVINGS_ACCOUNT,3826.72,Caixinha,2023-09-14 14:03:16,Ana Julia Santos,conservador,-262.70,2023-06-21 00:34:08,Outras saídas
4,BANK,SAVINGS_ACCOUNT,3826.72,Caixinha,2023-09-14 14:03:16,Ana Julia Santos,conservador,-179.44,2023-08-05 01:09:42,"Imposto, juros e multa"


In [145]:
merged_df.shape

(7569, 12)

Observe que a dimensão de transactions foi preservada! Muito bom, a união foi feita corretamente e agora temos, para cada transação, o nome do indivíduo que a realizou.

In [146]:
df = pd.merge(merged_df, transactions, on='account_id', how='inner')
df.head()

,id_x,type,subtype,balance,currency_code_x,name_x,person_id,account_id,date_x,id_y,name_y,suitability,id,currency_code_y,amount,date_y,category
0,1968bd13-dd25-4f88-bccc-5cff92d6811b,BANK,SAVINGS_ACCOUNT,3826.72,BRL,Caixinha,644f5d18-5bfa-45f9-8aa7-401c05b3b9f2,1968bd13-dd25-4f88-bccc-5cff92d6811b,2023-09-14 14:03:16,644f5d18-5bfa-45f9-8aa7-401c05b3b9f2,Ana Julia Santos,conservador,b7ebd81b-98f7-47ab-91f3-52bc80be57a6,BRL,-498.71,2023-07-20 03:57:57,Educação
1,1968bd13-dd25-4f88-bccc-5cff92d6811b,BANK,SAVINGS_ACCOUNT,3826.72,BRL,Caixinha,644f5d18-5bfa-45f9-8aa7-401c05b3b9f2,1968bd13-dd25-4f88-bccc-5cff92d6811b,2023-09-14 14:03:16,644f5d18-5bfa-45f9-8aa7-401c05b3b9f2,Ana Julia Santos,conservador,b8d83679-02ab-4241-b15d-11e9c0033864,BRL,-265.00,2023-07-23 05:17:06,Outras saídas
2,1968bd13-dd25-4f88-bccc-5cff92d6811b,BANK,SAVINGS_ACCOUNT,3826.72,BRL,Caixinha,644f5d18-5bfa-45f9-8aa7-401c05b3b9f2,1968bd13-dd25-4f88-bccc-5cff92d6811b,2023-09-14 14:03:16,644f5d18-5bfa-45f9-8aa7-401c05b3b9f2,Ana Julia Santos,conservador,99cab263-da5c-4706-9f2c-92cb779acaf7,BRL,-18.95,2023-06-26 05:54:25,Transporte
3,1968bd13-dd25-4f88-bccc-5cff92d6811b,BANK,SAVINGS_ACCOUNT,3826.72,BRL,Caixinha,644f5d18-5bfa-45f9-8aa7-401c05b3b9f2,1968bd13-dd25-4f88-bccc-5cff92d6811b,2023-09-14 14:03:16,644f5d18-5bfa-45f9-8aa7-401c05b3b9f2,Ana Julia Santos,conservador,5a78f1f0-f541-440f-9d3d-c8c21373ffc8,BRL,-262.70,2023-06-21 00:34:08,Outras saídas
4,1968bd13-dd25-4f88-bccc-5cff92d6811b,BANK,SAVINGS_ACCOUNT,3826.72,BRL,Caixinha,644f5d18-5bfa-45f9-8aa7-401c05b3b9f2,1968bd13-dd25-4f88-bccc-5cff92d6811b,2023-09-14 14:03:16,644f5d18-5bfa-45f9-8aa7-401c05b3b9f2,Ana Julia Santos,conservador,fc4e9552-bb41-4757-8c16-ff2c584ee01b,BRL,-179.44,2023-08-05 01:09:42,"Imposto, juros e multa"


In [147]:
df.shape

(400110, 17)

Perfeito! Os 3 datasets estão corretamente unidos e já podemos começar a explorar o dataset unido!

Irei renomear algumas features a fim de melhorar a interpretação e tornar a análise exploratória de dados mais fácil. Além disso, irei remover colunas com ids e currency_code pois elas não têm relevância para a análise e modelagem, uma vez que apresentam um valor único para cada observação. Por final irei remover colunas com o nome dos indivíduos pois, da mesma forma, não há relevância para a análise e modelagem, e a coluna account_date, dado que não tenho informações sobre o que ela representa e a coluna transaction_date será de maior utilidade para observar os padrões em transações ao longo do tempo, permitindo um foco maior no que importa.

In [148]:
print(df['currency_code_y'].unique())
print(df['currency_code_x'].unique())

['BRL']
['BRL']


In [149]:
df.columns

Index(['id_x', 'type', 'subtype', 'balance', 'currency_code_x', 'name_x',
       'person_id', 'account_id', 'date_x', 'id_y', 'name_y', 'suitability',
       'id', 'currency_code_y', 'amount', 'date_y', 'category'],
      dtype='object')

Removendo as colunas com um valor único para cada observação, a coluna contendo os nomes das pessoas e a coluna de datas para cada transação.

In [150]:
df.drop(columns=['id_x', 'person_id', 'account_id', 'id_y', 'id', 'currency_code_x', 'currency_code_y', 'name_y', 'date_x'], inplace=True)

Renomeando colunas.

In [151]:
df.rename(columns={'type': 'account_type', 'subtype': 'account_subtype', 'balance': 'account_balance', 'currency_code_x': 'account_currency_code', 'name_x': 'account_name', 'suitability': 'person_suitability', 'currency_code_y': 'transaction_currency_code', 'amount': 'transaction_amount', 'date_y': 'transaction_date', 'category': 'transaction_category'}, inplace=True)
df.head()

,account_type,account_subtype,account_balance,account_name,person_suitability,transaction_amount,transaction_date,transaction_category
0,BANK,SAVINGS_ACCOUNT,3826.72,Caixinha,conservador,-498.71,2023-07-20 03:57:57,Educação
1,BANK,SAVINGS_ACCOUNT,3826.72,Caixinha,conservador,-265.00,2023-07-23 05:17:06,Outras saídas
2,BANK,SAVINGS_ACCOUNT,3826.72,Caixinha,conservador,-18.95,2023-06-26 05:54:25,Transporte
3,BANK,SAVINGS_ACCOUNT,3826.72,Caixinha,conservador,-262.70,2023-06-21 00:34:08,Outras saídas
4,BANK,SAVINGS_ACCOUNT,3826.72,Caixinha,conservador,-179.44,2023-08-05 01:09:42,"Imposto, juros e multa"


### Dicionário de variáveis
1. account_type: Tipo de conta (BANK, CREDIT), categórica nominal.
2. account_subtype: Subtipo de conta (SAVINGS_ACCOUNT, CREDIT_CARD, CHECKING_ACCOUNT), categórica nominal.
3. account_balance: Saldo da conta, numérica contínua.
4. account_name: Nome da conta (Caixinha, Cartão de Crédito, Conta Corrente, Conta Remunerada, Poupança), categórica nominal.
5. person_suitability: Perfil do cliente (conservador, moderado-agressivo, moderado-conservador, moderado, agressivo), categórica ordinal.
6. transaction_amount: Total dispendido na transação, numérica contínua.
7. transaction_date: Data da transação, categórica nominal.
8. transaction_category: Finalidade da transação (Educação, Outras saídas, Transporte,
       Imposto, juros e multa, Investimento, Alimentação,
       Saúde, Recebido, Moradia, Compras, Lazer), categórica nominal.

In [154]:
df['transaction_category'].unique()

array(['Educação', 'Outras saídas', 'Transporte',
       'Imposto, juros e multa', 'Investimento', 'Alimentação', nan,
       'Saúde', 'Recebido', 'Moradia', 'Compras', 'Lazer'], dtype=object)